<a href="https://colab.research.google.com/github/rohith033/BTP/blob/main/Dqn_Traning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'shimmy'
!pip install stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
from osgeo import gdal
import numpy as np

# Load the raster file
raster = gdal.Open('/content/Depth (Max) (1).Terrain.needed_tif.tif')
band = raster.GetRasterBand(1)  # Assuming depth is in band 1
depth_data = band.ReadAsArray()

# Get geotransformation
geotransform = raster.GetGeoTransform()

In [ ]:
def get_flat_index_neighbors_with_diagonals(flat_index, num_rows, num_cols):
    i = flat_index // num_cols
    j = flat_index % num_cols
    neighbors = {}

    # Direct neighbors
    if i > 0 and (i-1) * num_cols + j<num_rows*num_cols:
        neighbors['top'] = (i-1) * num_cols + j
    if i < num_rows - 1 and (i+1) * num_cols + j<num_rows*num_cols:
        neighbors['bottom'] = (i+1) * num_cols + j
    if j > 0 and i * num_cols + (j-1) < num_rows*num_cols :
        neighbors['left'] = i * num_cols + (j-1)
    if j < num_cols - 1 and i * num_cols + (j+1) < num_rows*num_cols:
        neighbors['right'] = i * num_cols + (j+1)

    # Diagonal neighbors
    if i > 0 and j > 0 and (i-1) * num_cols + (j-1) < num_rows*num_cols:
        neighbors['top_left'] = (i-1) * num_cols + (j-1)
    if i > 0 and j < num_cols - 1 and (i-1) * num_cols + (j+1) <num_rows*num_cols:
        neighbors['top_right'] = (i-1) * num_cols + (j+1)
    if i < num_rows - 1 and j > 0 and (i+1) * num_cols + (j-1) < num_rows*num_cols:
        neighbors['bottom_left'] = (i+1) * num_cols + (j-1)
    if i < num_rows - 1 and j < num_cols - 1 and (i+1) * num_cols + (j+1)<num_rows*num_cols:
        neighbors['bottom_right'] = (i+1) * num_cols + (j+1)

    return neighbors

In [ ]:
num_rows, num_cols = depth_data.shape
lats = np.array([geotransform[3] + geotransform[5] * i for i in range(num_rows)])
longs = np.array([geotransform[0] + geotransform[1] * j for j in range(num_cols)])
# Create an empty list to store the data
data_matrix = []

# Iterate through each cell
for i in range(num_rows):
    for j in range(num_cols):
        lat = lats[i]
        lon = longs[j]
        depth = depth_data[i, j]
        data_matrix.append([lat, lon, depth])

# Optionally convert to a numpy array for better handling
data_matrix = np.array(data_matrix)
data_matrix[data_matrix == -9999] = 0

In [ ]:
total_states =  40
environments = []

In [ ]:
def make_env(flat_index,base_env,i,j):
  if i<0 or j<0 or i>8 or j>8 or base_env[i][j] != 0:
    return
  base_env[i][j] = data_matrix[flat_index][2]
  if base_env[i][j] == 0:
    base_env[i][j]  = 0.0001
  space = get_flat_index_neighbors_with_diagonals(flat_index,num_rows,num_cols)
  if 'top' in space:
    make_env(space['top'],base_env,i-1,j)
  if 'bottom' in space:
    make_env(space['bottom'],base_env,i+1,j)
  if 'left' in space:
    make_env(space['left'],base_env,i,j-1)
  if 'right' in space:
    make_env(space['right'],base_env,i,j+1)
  return

In [ ]:
for flat_index , depth_info in enumerate(data_matrix):
  if total_states == 0:
    break
  if depth_info[2] > 0.99:
    total_states-=1
    base_env = np.zeros((9,9))
    make_env(flat_index,base_env,4,4)
    environments.append(base_env)

In [ ]:
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback

class CustomFrozenLakeEnv(gym.Env):
    def __init__(self,depth_data):
        self.desc = ["CCCFFCCCC",
             "CFCFCFFFC",
             "CFCCCFHCC",
             "CFCCCFHCC",
             "CCCFFCCCC",
             "CFCCCFHCC",
             "CFFCFBHCC",
             "CFFFFFCCC",
             "CFFFCCCCC"]


        # Generate random depths between 0.5 and 5 for 'C' blocks
        self.depth_map = np.array(depth_data)
        self.colm = self.depth_map[0].shape[0]
        self.nS = 81
        self.nA = 4  # Define the number of actions (left, down, right, up)
        self.start_state = self._get_state(row=0, col=0)
        min_depth_point = np.unravel_index(np.argmin(self.depth_map), self.depth_map.shape)
        self.thersold = 0.2
        self.goal_state = self._get_state(row=min_depth_point[0], col=min_depth_point[1])
        self.state = None
        self.action_space = gym.spaces.Discrete(self.nA)  # Define the action space
        self.observation_space = gym.spaces.Discrete(self.nS)  # Define the observation space
        self.reset()

    def _get_state(self, row, col):
        """Converts row and column coordinates to a state index."""
        return row * 9 + col

    def reset(self):
        self.state = self.start_state
        return self._get_obs(self.state)

    def _get_obs(self, state):
        """Returns the observation (state index) for the current state."""
        return state

    def step(self, action):
        state = self.state
        reward = 0
        done = False

        # Probabilistic movement (same as standard FrozenLake)
        if np.random.rand() < 0.1:
            state = (state + np.random.randint(-1, 2, size=1)) % self.nS
        else:
            new_state = state + action
            if 0 <= new_state < self.nS:
                state = new_state
        # Update depth reward
        desc = np.array(list(''.join(self.desc))).reshape(9,9)
        crow = int(state / self.colm)
        ccol =  state % self.colm
        reward += -(((self.depth_map[crow,ccol]+1)/10)*reward)
        if desc[crow][ccol] == 'F':
          reward += -(reward*0.1)
        if desc[crow][ccol] == 'B':
          reward += -(reward*0.1)
        if self.depth_map[crow,ccol] <= self.thersold:
          done = True
        # Check for terminal states (hole or goal)
        if state == self.start_state or state == self.goal_state:
            done = True
        self.state = state
        return self._get_obs(state), reward, done, {}
    def render(self, mode='human'):
        """Renders a text-based representation of the environment."""
        if mode == 'human':
            desc = np.array(list(''.join(self.desc))).reshape(9,9)
            for i in range(9):
                for j in range(9):
                    if self.state == self._get_state(i, j):
                        desc[i, j] = desc[i, j].lower()  # Highlight current state
                print(''.join(desc[i]))
        else:
            super().render(mode=mode)  # Default rendering for compatibility (optional)
# Example usage
env = CustomFrozenLakeEnv(environments[0])
state = env.reset()
done = False
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
# Save the trained model
model.save("trained_dqn")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2776     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.992    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1599     |
|    time_elapsed     | 0        |
|    total_timesteps  | 8        |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1        |
|    ep_rew_mean      | 0      

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
<ipython-input-64-9ba81abcf986>:60: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  crow = int(state / self.colm)
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/monitor.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.rewards.append(float(reward))
/usr/local/lib/python3.1

Streaming output truncated to the last 5000 lines.
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.63e-06 |
|    n_updates        | 1722     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.41     |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2252     |
|    fps              | 501      |
|    time_elapsed     | 13       |
|    total_timesteps  | 6996     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.81e-06 |
|    n_updates        | 1723     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.41     |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 2256     |
|   

array([[2.73033171, 3.42724352, 2.43379069, 4.91589059, 0.5563944 ,
        3.86391652],
       [1.68936224, 4.37137733, 4.50495599, 4.56586023, 1.37073306,
        1.64377439],
       [1.12076045, 4.78594611, 2.81792076, 3.5085163 , 4.41348353,
        2.56888276],
       [4.50309335, 4.54345797, 0.9106546 , 2.40940781, 1.40335397,
        4.25389508],
       [4.44283339, 2.70522361, 3.15637284, 1.01771341, 1.80229255,
        4.48547471],
       [1.68219801, 1.22015471, 1.70614777, 2.7372144 , 2.01864573,
        2.51864987],
       [3.41272374, 4.79101323, 4.5480453 , 4.65366123, 2.98660543,
        1.81172613],
       [3.80310458, 3.68140571, 0.79788027, 0.89207727, 3.34964312,
        2.1082193 ]])